## Packages

In [1]:
import pandas as pd
import re
import string
import nltk
from sklearn.manifold import TSNE
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

## Functions

In [2]:
def transformDocument(df, column_name, language):
    stop_words = usingStopwords(language)
    # 1. Aplicar preprocessamento nos títulos e textos completos
    if language == 'pt':
        # Substituir símbolos importantes
        df[column_name] = df[column_name].map(lambda s: s.replace('-feira', ''))
        df[column_name] = df[column_name].map(lambda s: s.replace('+', 'mais '))
        df[column_name] = df[column_name].map(lambda s: s.replace('-', 'menos '))
        df[column_name] = df[column_name].map(lambda s: s.replace('%', ' por cento'))
        df[column_name] = df[column_name].map(lambda s: removeStopwords(s, stop_words))

    elif language == 'en':
        df[column_name] = df[column_name].map(lambda s: s.replace('-', 'less'))
        df[column_name] = df[column_name].map(lambda s: s.replace('+', 'plus '))
        df[column_name] = df[column_name].map(lambda s: s.replace('%', ' percent'))
        df[column_name] = df[column_name].map(lambda s: removeStopwords(s, stop_words))

    else:
        pass

    df[column_name] = df[column_name].map(lambda s: s.replace('R$', ''))
    df[column_name] = df[column_name].map(lambda s: s.replace('U$', ''))
    df[column_name] = df[column_name].map(lambda s: s.replace('US$', ''))
    df[column_name] = df[column_name].map(lambda s: s.replace('S&P 500', 'spx'))

    # Transformar em String e Letras Minúsculas nas Mensagens
    df[column_name] = df[column_name].map(lambda s:
                                              normalizarString(s))

    # Remover Pontuações
    # Remover Pontuações
    df[column_name] = df[column_name].map(lambda s: s.translate(str.maketrans('', '', string.punctuation)))

    # Remover Emojis
    df[column_name] = df[column_name].map(lambda s: removeEmojis(s))

    # Quebras de Linha desnecessárias
    df[column_name] = df[column_name].map(lambda s: s.replace('\n', ' '))

    # Remover aspas duplas
    df[column_name] = df[column_name].map(lambda s: s.replace('\"', ''))
    df[column_name] = df[column_name].map(lambda s: s.replace('“', ''))
    df[column_name] = df[column_name].map(lambda s: s.replace('”', ''))

    # Remover valores
    df[column_name] = df[column_name].map(lambda s: removeValores(s))

    # Espaços desnecessários
    df[column_name] = df[column_name].map(lambda s: s.strip())
    return df



def removeEmojis(sentence):
    "Remoção de Emojis nas mensagens de texto."

    # Padrões dos Emojis
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u'\U00010000-\U0010ffff'
                               u"\u200d"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\u3030"
                               u"\ufe0f"
                               "]+", flags=re.UNICODE)

    return emoji_pattern.sub(r'', sentence)

def removeValores(sentence):
    new_sentece = ''

    for token in sentence.split():
        if token.isdigit():
            token = '<NUM>'
        new_sentece += ' {}'.format(token)

    return new_sentece

def usingStopwords(language):
    stop_words = []

    nltk.download('stopwords')

    if language == 'pt':
        stop_words = nltk.corpus.stopwords.words('portuguese')
    elif language == 'en':
        stop_words = nltk.corpus.stopwords.words('english')

    return stop_words

def removeStopwords(text, stop_words):
    tokens = []
    for word in text.split():
        if word not in stop_words:
            tokens.append(word)

    text = ' '.join(tokens)
    return text

def normalizarString(text):
    """
    Função para retirar acentuações e converter para minúscula
    :param text:
    :return text_normalizado
    """
    import unicodedata

    text = unicodedata.normalize('NFD', text).encode('ascii', 'ignore').decode("utf-8")
    return str(text.lower())

## Load dataset

In [3]:
df = pd.read_csv('../assets/df.csv')
df

,title,tags,link,label
0,Dividendos: As 10 ações preferidas por 11 anal...,"['Carteira Recomendada', 'Comprar ou vender?',...",https://www.moneytimes.com.br/dividendos-as-10...,1
1,"Itaú, JHSF e Banco do Brasil: confira a agenda...","['Alper (APER3)', 'balanços trimestrais', 'Bol...",https://www.suno.com.br/noticias/itau-jhsf-ban...,2
2,"Destaques de Empresas: Petrobras (PETR4), JBS ...","['Arezzo (ARZZ3)', 'Cia Hering (HGTX3)', 'dest...",https://www.suno.com.br/noticias/destaques-de-...,0
3,Itaú Unibanco (ITUB4) vai pagar proventos no c...,"['Bancos', 'Dividendos', 'Empresas', 'Itaú Uni...",https://www.moneytimes.com.br/itau-unibanco-it...,2
4,Petrobras (PETR4) elege novo conselho com maio...,"['Ações', 'Adriano Pires', 'Combustíveis', 'Em...",https://www.moneytimes.com.br/petrobras-petr4-...,0
...,...,...,...,...
11951,Petrobras guia nova máxima do Ibovespa em sess...,"['Ações', 'Empresas', 'Ibovespa', 'Leilão', 'M...",https://www.moneytimes.com.br/petrobras-guia-n...,0
11952,Petrobras estende vencimento de empréstimo com...,"['Empresas', 'Petrobras']",https://www.moneytimes.com.br/petrobras-estend...,0
11953,Petrobras: percepção de risco sobre política d...,"['Ações', 'Combustíveis', 'Commodities', 'Comp...",https://www.moneytimes.com.br/petrobras-percep...,0
11954,BNDES espera vender cerca de R$ 2 bi em debênt...,"['BNDES', 'BNDESPar', 'debêntures', 'Vale (VAL...",https://www.suno.com.br/noticias/bndes-venda-d...,1


In [4]:
df

,title,tags,link,label
0,Dividendos: As 10 ações preferidas por 11 anal...,"['Carteira Recomendada', 'Comprar ou vender?',...",https://www.moneytimes.com.br/dividendos-as-10...,1
1,"Itaú, JHSF e Banco do Brasil: confira a agenda...","['Alper (APER3)', 'balanços trimestrais', 'Bol...",https://www.suno.com.br/noticias/itau-jhsf-ban...,2
2,"Destaques de Empresas: Petrobras (PETR4), JBS ...","['Arezzo (ARZZ3)', 'Cia Hering (HGTX3)', 'dest...",https://www.suno.com.br/noticias/destaques-de-...,0
3,Itaú Unibanco (ITUB4) vai pagar proventos no c...,"['Bancos', 'Dividendos', 'Empresas', 'Itaú Uni...",https://www.moneytimes.com.br/itau-unibanco-it...,2
4,Petrobras (PETR4) elege novo conselho com maio...,"['Ações', 'Adriano Pires', 'Combustíveis', 'Em...",https://www.moneytimes.com.br/petrobras-petr4-...,0
...,...,...,...,...
11951,Petrobras guia nova máxima do Ibovespa em sess...,"['Ações', 'Empresas', 'Ibovespa', 'Leilão', 'M...",https://www.moneytimes.com.br/petrobras-guia-n...,0
11952,Petrobras estende vencimento de empréstimo com...,"['Empresas', 'Petrobras']",https://www.moneytimes.com.br/petrobras-estend...,0
11953,Petrobras: percepção de risco sobre política d...,"['Ações', 'Combustíveis', 'Commodities', 'Comp...",https://www.moneytimes.com.br/petrobras-percep...,0
11954,BNDES espera vender cerca de R$ 2 bi em debênt...,"['BNDES', 'BNDESPar', 'debêntures', 'Vale (VAL...",https://www.suno.com.br/noticias/bndes-venda-d...,1


## Preprocessing dataset

In [5]:
df_pp = df.copy()
df_pp = transformDocument(df_pp, 'title', 'pt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Guilherme\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
df_pp

,title,tags,link,label
0,dividendos as <NUM> acoes preferidas <NUM> ana...,"['Carteira Recomendada', 'Comprar ou vender?',...",https://www.moneytimes.com.br/dividendos-as-10...,1
1,itau jhsf banco brasil confira agenda resultad...,"['Alper (APER3)', 'balanços trimestrais', 'Bol...",https://www.suno.com.br/noticias/itau-jhsf-ban...,2
2,destaques empresas petrobras petr4 jbs jbss3 h...,"['Arezzo (ARZZ3)', 'Cia Hering (HGTX3)', 'dest...",https://www.suno.com.br/noticias/destaques-de-...,0
3,itau unibanco itub4 vai pagar proventos comeco...,"['Bancos', 'Dividendos', 'Empresas', 'Itaú Uni...",https://www.moneytimes.com.br/itau-unibanco-it...,2
4,petrobras petr4 elege novo conselho maior part...,"['Ações', 'Adriano Pires', 'Combustíveis', 'Em...",https://www.moneytimes.com.br/petrobras-petr4-...,0
...,...,...,...,...
11951,petrobras guia nova maxima ibovespa sessao meg...,"['Ações', 'Empresas', 'Ibovespa', 'Leilão', 'M...",https://www.moneytimes.com.br/petrobras-guia-n...,0
11952,petrobras estende vencimento emprestimo banco ...,"['Empresas', 'Petrobras']",https://www.moneytimes.com.br/petrobras-estend...,0
11953,petrobras percepcao risco sobre politica preco...,"['Ações', 'Combustíveis', 'Commodities', 'Comp...",https://www.moneytimes.com.br/petrobras-percep...,0
11954,bndes espera vender cerca <NUM> bi debentures ...,"['BNDES', 'BNDESPar', 'debêntures', 'Vale (VAL...",https://www.suno.com.br/noticias/bndes-venda-d...,1


## Exporting dataset preprocessed for word2vec and EDA

In [8]:
df_pp.to_csv('../assets/df_pp.csv', index=False)